In [1]:
def eliminate_implication(kb):
    new_kb = []
    for statement in kb:
        if ('∀' in statement or '∃' in statement) and '→' in statement:
            quantifier_part, rest = statement.split('(', 1)
            variable, inner_statement = rest.split(')', 1)
            variable = variable + ')'
            inner_statement = inner_statement.replace('→', '∨')
            inner_statement = inner_statement.strip()
            variable = '¬' + variable

            new_statement = f"{quantifier_part}({variable} {inner_statement}"
            new_kb.append(new_statement)
        elif '→' in statement:
            parts = statement.split('→')
            new_kb.append(f"(¬{parts[0].strip()} ∨ {parts[1].strip()})")
        else:
            new_kb.append(statement)
    return new_kb

statement = ["∀x(Human(x) → Mortal(x))"]
print(eliminate_implication(statement))

['∀x(¬Human(x) ∨ Mortal(x))']


In [2]:
def demorgans(kb):
    new_kb = []
    
    for expression in kb:
        if "¬(" in expression:
            # Find the index of the first occurrence of "¬("
            not_index = expression.index("¬(")
            
            # Find the corresponding closing parenthesis
            open_count = 0
            close_index = -1
            for i in range(not_index + 1, len(expression)):
                if expression[i] == "(":
                    open_count += 1
                elif expression[i] == ")":
                    if open_count == 1:                                                                                 #from 0 to 1
                        close_index = i
                        break
                    else:
                        open_count -= 1

            if close_index != -1:
                # Extract the expression inside the parentheses
                inside_expression = expression[not_index + 2:close_index]
           

                # Apply De Morgan's Law
                new_expression = expression[:not_index] + "("
                variable = ""
                for z in range(0, len(inside_expression)):
                    
                    if inside_expression[z] == "^":
                        if variable != "":
                            new_expression += "¬" + variable
                            variable = ""
                        new_expression += "∨"

                    elif inside_expression[z] == "∨":
                        if variable != "":
                            new_expression += "¬" + variable
                            variable = ""
                        new_expression += "^"

                    elif inside_expression[z] == "∀":
                        new_expression += "∃" + inside_expression[z + 1]
                        if variable != "":
                            new_expression += "¬" + variable
                            variable = ""

                    elif inside_expression[z]== "∃":
                        new_expression += "∀" + inside_expression[z + 1]
                        if variable != "":
                            new_expression += "¬" + variable
                            variable = ""   
                        

                    elif inside_expression[z] == "¬" or inside_expression[z]  == " ":
                        if variable != "":
                            new_expression += "¬" + variable
                            variable = ""
                        new_expression += inside_expression[z]

                    else:
                        if inside_expression[z-1] == "∀" or inside_expression[z-1] == "∃":
                            continue
                        else:
                            variable += inside_expression[z]
                        
                if variable != "":
                    new_expression += "¬" + variable

                new_expression += ")" + expression[close_index + 1:]
                new_kb.append(new_expression)
                

    
    if new_kb != []:
        return new_kb
    else:
        return kb
    
statement = ["¬(∀xP(x) ∨ ∃y¬Q(y))"]
print(demorgans(statement))

['(∃x¬P(x) ^ ∀y¬¬Q(y))']


In [3]:
def double_not(kb):
    new_kb = []
    for i in range(len(kb)):
        new_kb.append(kb[i].replace("¬¬", ""))
    return new_kb
 

statement = ["¬(∀xP(x) ∨ ∃y¬¬Q(y))"]
print(double_not(statement))

['¬(∀xP(x) ∨ ∃yQ(y))']


In [4]:
def standardize(kb):
     new_kb = []
     for expression in kb:
          symbols = []
          expression_holder = expression
          
          for i in range (0,len(expression_holder)-1):
               counter = 1
               new_symbol = ''

               if expression_holder[i] == "∃" or  expression_holder[i] == "∀":
                         new_symbol_code = -1
                    #We see if the symbol is not dublicated
                         if expression_holder[i+1] not in symbols:
                               symbols.append(expression_holder[i+1])
                    #If dublicated
                         else:
                              new_expression = ""
                    #getting new symbol
                              while expression_holder[i+1] in symbols:
                                   new_symbol_code = ord(expression_holder[i+1]) + counter
                                   counter = counter + 1
                                   if new_symbol_code != -1:
                                        new_symbol = chr(new_symbol_code)
                                        symbols.append(new_symbol)
                                        break
                    #applying the new symbol
                              for z in range (0, i+1):
                                   new_expression = new_expression + expression_holder[z]
                              for j in range (i+1, len(expression_holder)):
                                  if(expression_holder[i+1] == expression_holder[j]):
                                        new_expression = new_expression + new_symbol
                                  else:
                                        new_expression = new_expression + expression_holder[j]
                    #changing the expression after applying the new symbol
                              expression_holder = new_expression
                                        
                            
                                        
                                   
                        
          
          new_kb.append(expression_holder)

     return new_kb   


statement = ["(∀a P(a)) ∨ (∃a Q(a)) ∨ (∃a Q(a)) ∨ (∃a Q(a))"]
print(standardize(statement))   

['(∀a P(a)) ∨ (∃b Q(b)) ∨ (∃c Q(c)) ∨ (∃d Q(d))']


In [5]:
def move_quantifiers(statement):
    
    expressions=""
    temp = statement
    for i in range(len(statement)):
        if statement[i]=='∀' or statement[i]=='∃':
            expressions+=statement[i]
            expressions+=statement[i+1]
            new = temp.replace(statement[i:i+2],"")
            temp = new

    statement = expressions + temp
    
    return statement

statement = '∀x P(x) ∨ ∃y Q(y) & ∃z Q(z)'
print(move_quantifiers(statement))

∀x∃y∃z P(x) ∨  Q(y) &  Q(z)


In [6]:
import random
chars = ['H','M','N','C','D']

def skolemization(statement):
    temp = statement
    if '∃' not in statement:
        return statement
    elif statement[0]=='∃':
        temp = temp.replace(statement[0:2],"")
        temp = temp.replace(statement[1],chars[random.randint(0, 4)])
    elif statement[0]=='∀':
        var1 = statement[1]
        var2 = statement[3]
        toReplace = "F(" + var1 + ")"
        temp = temp.replace(statement[2:5]," ")
        temp = temp.replace(statement[3],toReplace)

    return temp


test = "∀x∃y P(x) ∨ Q(y)"
print(skolemization(test))

∀x P(x) ∨ Q(F(x))


In [7]:
def drop_universal_quantifiers(statement):
    temp = statement
    while temp[0]=='∀':
        temp = temp.replace(temp[0:2],"")
    
    return temp


statement = '∀x∀y∀z (P(x) & P(y) & P(z))'
print(drop_universal_quantifiers(statement))

 (P(x) & P(y) & P(z))


In [8]:
def cnf(kb):
    new_kb = []

    new_kb = eliminate_implication(kb)
    print("After implication elimination: ")
    print(new_kb)

    new_kb = demorgans(new_kb)
    print("After Demorgans: ")
    print(new_kb)

    new_kb = double_not(new_kb)
    print("After double not elimination: ")
    print(new_kb)

    new_kb = standardize(new_kb)
    print("After standardization : ")
    print(new_kb)

    new_kb = [move_quantifiers(elem) for elem in new_kb]
    print("After prenex form : ")
    print(new_kb)

    new_kb = [skolemization(elem) for elem in new_kb]
    print("After skolemization : ")
    print(new_kb)

    new_kb = [drop_universal_quantifiers(elem) for elem in new_kb]
    print("After dropping universal quantifiers : ")
    print(new_kb)

    return new_kb

In [9]:
def rename_vriables(kb):
    names = ['W','A','X','C','M']
    cnt = 0
    res = []
    for statement in kb:
        temp = statement
        temp = temp[1:]
        temp = temp[:-1]
        for i in range(len(temp)):
            if temp[i]=='(':
                temp = temp.replace(temp[i+1],names[cnt])
                res.append(temp)
                cnt+=1
                break
    out = []
    for statement in res:
        char=''
        for k in range(len(statement)):
            if statement[k]=='(':
                char = statement[k+1]
                break
        changed = ""
        for l in range(len(statement)):
            if statement[l]=='(':
                changed = statement.replace(statement[l+1],char)
        out.append(changed)
    return out

test = ["(¬Human(x) ∨ Mortal(c))", "(¬Parent(x, Child(x) ))", "(¬King(x) ∨ Ruler(x))"]
print(rename_vriables(test))

['¬Human(W) ∨ Mortal(W)', '¬Parent(A, Child(A) )', '¬King(X) ∨ Ruler(X)']


In [10]:
test = ["∀x(Student(x) → Attend(x))","∃y∀x(Doctor(x) ∨ Engineer(y))","∀x(Student(z) → ¬¬Excellent(z))","∀z(Exam(z)) ∨ ∀y (Depressed(y))"]

cnf1 = cnf(test)
res = rename_vriables(cnf1)
print("Final output: ")
print(res)

After implication elimination: 
['∀x(¬Student(x) ∨ Attend(x))', '∃y∀x(Doctor(x) ∨ Engineer(y))', '∀x(¬Student(z) ∨ ¬¬Excellent(z))', '∀z(Exam(z)) ∨ ∀y (Depressed(y))']
After Demorgans: 
['∀x(¬Student(x) ∨ Attend(x))', '∃y∀x(Doctor(x) ∨ Engineer(y))', '∀x(¬Student(z) ∨ ¬¬Excellent(z))', '∀z(Exam(z)) ∨ ∀y (Depressed(y))']
After double not elimination: 
['∀x(¬Student(x) ∨ Attend(x))', '∃y∀x(Doctor(x) ∨ Engineer(y))', '∀x(¬Student(z) ∨ Excellent(z))', '∀z(Exam(z)) ∨ ∀y (Depressed(y))']
After standardization : 
['∀x(¬Student(x) ∨ Attend(x))', '∃y∀x(Doctor(x) ∨ Engineer(y))', '∀x(¬Student(z) ∨ Excellent(z))', '∀z(Exam(z)) ∨ ∀y (Depressed(y))']
After prenex form : 
['∀x(¬Student(x) ∨ Attend(x))', '∃y∀x(Doctor(x) ∨ Engineer(y))', '∀x(¬Student(z) ∨ Excellent(z))', '∀z∀y(Exam(z)) ∨  (Depressed(y))']
After skolemization : 
['∀x(¬Student(x) ∨ Attend(x))', '∀x(Doctor(x) ∨ Engineer(N))', '∀x(¬Student(z) ∨ Excellent(z))', '∀z∀y(Exam(z)) ∨  (Depressed(y))']
After dropping universal quantifiers : 
['(¬